In [1]:
!pip install langchain-chroma==0.1.0
!pip install langchainhub==0.1.15
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.4 MB/s

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/utils/logging.py", line 29, in <module>
^C
^C
^C


In [1]:
#Enter API Tokens
from getpass import getpass
OPENAI_KEY=getpass("Enter your API token here: ")

Enter your API token here: ··········


In [2]:
import os
os.environ['OPENAI_API_KEY']=OPENAI_KEY

In [3]:
import gzip
import json
import requests
import sys
import os
from tqdm import tqdm

# Load the Wikipedia data

In [4]:
def http_get(url, path) -> None:
    """
    Downloads a URL to a given path on disk
    """
    if os.path.dirname(path) != "":
        os.makedirs(os.path.dirname(path), exist_ok=True)

    req = requests.get(url.strip(), stream=True)
    if req.status_code != 200:
        print("Exception when trying to download {}. Response {}".format(url, req.status_code), file=sys.stderr)
        req.raise_for_status()
        return

    download_filepath = path + "_part"
    with open(download_filepath, "wb") as binary:
        content_length = req.headers.get("Content-Length")
        total = int(content_length) if content_length is not None else None
        progress = tqdm(unit="B", total=total, unit_scale=True)
        for chunk in req.iter_content(chunk_size=1024):
            if chunk:
                progress.update(len(chunk))
                binary.write(chunk)  # <- fixed name here

    os.rename(download_filepath, path)
    progress.close()

# Call the function
wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'
http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

100%|██████████| 50.2M/50.2M [00:13<00:00, 3.85MB/s]


In [5]:
import gzip
import json
wikipedia_filepath = "simplewiki-2020-11-01.jsonl.gz"
passages=[]
with gzip.open(wikipedia_filepath,"rt",encoding='utf8') as fIn:
  for line in fIn:
    data=json.loads(line.strip())
    passages.append(data['paragraphs'][0])
print("passages: ",len(passages))


passages:  169597


In [6]:
passages

['Ted Cassidy (July 31, 1932 - January 16, 1979) was an American actor. He was best known for his roles as Lurch and Thing on "The Addams Family".',
 'Aileen Carol Wuornos Pralle (born Aileen Carol Pittman; February 29, 1956\xa0– October 9, 2002) was an American serial killer. She was born in Rochester, Michigan. She confessed to killing six men in Florida and was executed in Florida State Prison by lethal injection for the murders. Wuornos said that the men she killed had raped her or tried to rape her while she was working as a prostitute.',
 "A crater is a round dent on a planet. They are usually shaped like a circle or an oval. They are usually made by something like a meteor hitting the surface of a planet. Underground activity such as volcanoes or explosions can also cause them but it's not as likely.",
 'Store has several meanings:',
 'Chinese New Year, known in China as the SpringFestival and in Singapore as the LunarNewYear, is a holiday on and around the new moon on the first

In [9]:
passages=[passage for passage in passages for x in ['washington','river','modi'] if x in passage.lower().split()]

In [10]:
len(passages)

8325

In [11]:
passages[0]

'Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'

In [12]:
passages[1]

'Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'

In [13]:
passages[3]

'The Szczecin Lagoon (Polish name) or Stettin Lagoon (German name) or Lagoon of the Oder River is an inland water basin found in the southwestern part of the Baltic Sea. It is shared by Germany and Poland. These are the towns that can be found at the Szczecin Lagoon: Świnoujście, Ueckermünde, Wolin (town) and Nowe Warpno.'

In [14]:
passages[2]

'Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'

In [15]:
passages[87]

'Yorktown is a city in Virginia. It is the county seat of York County, though it only has 220 people living there. It is most known for the Battle of Yorktown, between American/French troops under George Washington and Jean Rochambeau, and British troops under Charles Cornwallis.'

# **Load OpenAI LLMs**

In [17]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.0 requires langchain-core<0.2.0,>=0.1.40, but you have langchain-core 0.3.59 which is incompatible.


# Text Generation model (at the time of output geneartion)

In [18]:
from langchain_openai import ChatOpenAI
chatgpt=ChatOpenAI(model_name='gpt-4o-mini',temperature=0)

# Text Embedding (non numeric to numeric ) - at the time of converting chunking to chroma db

In [19]:
from langchain_openai import OpenAIEmbeddings
openai_embed_model=OpenAIEmbeddings(model="text-embedding-3-small")

# Chunking and Chroma db - we will be using Recursive Character Text Splitter and Chromadb

In [21]:
 from langchain.text_splitter import RecursiveCharacterTextSplitter
 from langchain_chroma import Chroma

In [22]:
from langchain.docstore.document import Document
docs=[Document(page_content=doc) for doc in passages]

In [26]:
# import os

# # Get the current working directory
# current_dir = os.getcwd()

# print("Current Directory:", current_dir)


Current Directory: /content


In [23]:
docs

[Document(metadata={}, page_content='Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'),
 Document(metadata={}, page_content='Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'),
 Document(metadata={}, page_content='Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is anot

In [24]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=300)
chunked_docs= splitter.split_documents(docs)

In [25]:
chunked_docs[:3]

[Document(metadata={}, page_content='Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'),
 Document(metadata={}, page_content='Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is another town called Nowe Warpno.'),
 Document(metadata={}, page_content='Police is a town in Poland in West Pomeranian Voivodeship. As of 2007, 34,220 people live there. The town is located on the river Oder, near the border with Germany. It is one of the largest sea ports in Poland. Police borders with Szczecin. In the Police area, there is anot

# Create Vector DB and Retriver

# if you have already created wiki_db in theprevious session then just load the DB and Do NOT run the folowing code to create database again , ignore this when running juyter notebook or colab

In [27]:
chroma_db=Chroma.from_documents(documents=chunked_docs,collection_name='wiki_db',embedding=openai_embed_model,
                                collection_metadata={"hnsw:space": "cosine"},persist_directory='./wiki_db')

In [28]:
chroma_db

In [29]:
similarity_retriever=chroma_db.as_retriever(search_type="similarity_score_threshold",search_kwargs={'k':5,"score_threshold":0.3})

In [30]:
similarity_retriever.invoke("what is the capital of india")

[Document(metadata={}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once the center of industry and education. However, it has witnessed political violence and economic problems since 1954. Since 2000, Kolkata has grown due to economic growth. Like other metropolitan cities in India, Kolkata struggles with poverty, pollution and traffic congestion."),
 Document(metadata={}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai

# Building a QA RAG Chain

In [31]:
from langchain_core.prompts import ChatPromptTemplate
prompt=""" You are an assistant for question-answering tasks.
use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Keep your answer upto 5 lines unless the user asks for more information.
Question : {question}
Context : {context}
Answer:
"""

prompt_template=ChatPromptTemplate.from_template(prompt)

# New Langchain syntax for Rag chain-using LCEL

In [32]:
from langchain_core.runnables import RunnablePassthrough
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)
qa_rag_chain=({"context":(similarity_retriever|format_docs),"question":RunnablePassthrough()}|prompt_template|chatgpt)

In [33]:
query="what is capital of India?"
result=qa_rag_chain.invoke(query)
print(result.content)

The capital of India is New Delhi.


In [34]:
query="who is sumit "
result=qa_rag_chain.invoke(query)
print(result.content)

I don't know.


In [35]:
query="who is subhas chandra  bose "
result=qa_rag_chain.invoke(query)
print(result.content)

Subhas Chandra Bose was an Indian nationalist leader who played a significant role in the Indian independence movement against British rule. He is best known for his leadership of the Indian National Army (INA) and his efforts to seek international support for India's independence. Bose was a prominent figure in the Indian National Congress and advocated for complete independence from British rule. He is often remembered for his slogan "Give me blood, and I shall give you freedom."


In [36]:
query="who is subhas handra bose "
result=qa_rag_chain.invoke(query)
print(result.content)

Subhas Chandra Bose was an Indian nationalist leader who played a significant role in the struggle for India's independence from British rule. He was a prominent figure in the Indian National Congress and later formed the Forward Bloc. Bose is best known for his efforts to seek international support against British colonialism and for leading the Indian National Army (INA) during World War II. His slogan "Give me blood, and I shall give you freedom" inspired many in the fight for independence.
